# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
df = pd.read_csv("output_data/cities.csv")
df.head()

,city,latitude,longitude,weather,temperature,humidity,pressure,wind_speed
0,maykor,59.007140,55.886933,Snow,33.94,98,1016,5.23
1,butaritari,2.920538,172.918863,Clouds,81.88,71,1008,19.28
2,rikitea,-23.122954,-134.968014,Clouds,73.44,85,1017,9.26
3,pangnirtung,66.148020,-65.717283,Clouds,2.59,97,1009,2.57
4,kaitangata,-46.281739,169.846414,Clear,55.98,57,1027,11.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
data = df.loc[:, ["latitude", "longitude"]]

figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '0px',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type='SATELLITE')

heat = gmaps.heatmap_layer(data, weights=df.humidity)
fig.add_layer(heat)

fig

Figure(layout=FigureLayout(border='0px', height='600px', margin='0 auto 0 auto', padding='1px', width='800px')…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
mask = (df.wind_speed < 10) & (df.temperature < 80) & (df.temperature > 70) & (df.weather == "Clear")
hotel_df = df.loc[mask].reset_index(drop=True)
hotel_df

,city,latitude,longitude,weather,temperature,humidity,pressure,wind_speed
0,abu dhabi,24.453835,54.377401,Clear,78.82,53,1014,4.61
1,tsihombe,-25.318547,45.484057,Clear,72.41,80,1014,5.35
2,nampula,-15.119406,39.261589,Clear,71.62,83,1014,9.22
3,salalah,17.017495,54.101092,Clear,73.47,53,1016,5.75
4,ibra,22.689588,58.546979,Clear,70.86,65,1017,2.28
5,atar,20.518194,-13.054393,Clear,77.61,16,1014,5.64
6,carutapera,-1.201664,-46.020562,Clear,74.50,88,1013,4.94
7,beloha,-25.173662,45.061040,Clear,71.22,87,1014,3.83
8,abu samrah,24.200574,55.399052,Clear,79.70,37,1016,5.53
9,reefton,-42.115938,171.863456,Clear,73.36,35,1022,5.79


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotels = []

for indx, row in hotel_df.iterrows():
    lat = row.latitude
    lng = row.longitude

    target_coordinates = f"{lat}, {lng}"
    target_search = "hotel"
    target_radius = 5000
    target_type = "lodging"

    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_url, params=params)
    data = response.json()
    
    if (len(data["results"]) > 0):
        hotel = data["results"][0]["name"]
        address = data["results"][0]["vicinity"]
        rating = data["results"][0]["rating"]
        user_ratings_total = data["results"][0]["user_ratings_total"]
    else:
        hotel = None
        address = None
        rating = np.nan
        user_ratings_total = np.nan
        
    hotel = {"hotel" : hotel,
            "address": address,
            "rating": rating,
            "user_ratings_total": user_ratings_total}
    
    hotels.append(hotel)

In [9]:
df2 = pd.DataFrame(hotels)
df2.head()

,hotel,address,rating,user_ratings_total
0,Sheraton Khalidiya Hotel,"Zayed The First Street, Opposite KM Trading",3.8,2966.0
1,Littoral Hôtel,Tsihombe Tsihombe Androy,5.0,1.0
2,Grand Plaza Hotel,"V7M7+P76, Nampula",4.5,785.0
3,Al Baleed Resort Salalah by Anantara,"PC 214, Al Mansurah Street, Al Baleed, Salalah",4.7,1406.0
4,Capital O 141 Golden Rays Hotel,Ibra,3.9,42.0


In [12]:
hotel_full = pd.merge(hotel_df, df2, left_index=True, right_index=True)
hotel_full

,city,latitude,longitude,weather,temperature,humidity,pressure,wind_speed,hotel,address,rating,user_ratings_total
0,abu dhabi,24.453835,54.377401,Clear,78.82,53,1014,4.61,Sheraton Khalidiya Hotel,"Zayed The First Street, Opposite KM Trading",3.8,2966.0
1,tsihombe,-25.318547,45.484057,Clear,72.41,80,1014,5.35,Littoral Hôtel,Tsihombe Tsihombe Androy,5.0,1.0
2,nampula,-15.119406,39.261589,Clear,71.62,83,1014,9.22,Grand Plaza Hotel,"V7M7+P76, Nampula",4.5,785.0
3,salalah,17.017495,54.101092,Clear,73.47,53,1016,5.75,Al Baleed Resort Salalah by Anantara,"PC 214, Al Mansurah Street, Al Baleed, Salalah",4.7,1406.0
4,ibra,22.689588,58.546979,Clear,70.86,65,1017,2.28,Capital O 141 Golden Rays Hotel,Ibra,3.9,42.0
5,atar,20.518194,-13.054393,Clear,77.61,16,1014,5.64,نزل البحر والصحراء,Unnamed Road,3.9,11.0
6,carutapera,-1.201664,-46.020562,Clear,74.50,88,1013,4.94,Center Hotel,"R. Dep. Manoel Ribeiro, 571 - Centro, Carutapera",4.2,54.0
7,beloha,-25.173662,45.061040,Clear,71.22,87,1014,3.83,None,None,NaN,NaN
8,abu samrah,24.200574,55.399052,Clear,79.70,37,1016,5.53,Cataract Resort,6C7W+7CV,3.8,138.0
9,reefton,-42.115938,171.863456,Clear,73.36,35,1022,5.79,Dawsons Hotel,"74 Broadway, Reefton",4.3,258.0


In [13]:
infos = []

for indx, row in hotel_full.iterrows():
    info_box_template = f"""<dl>
                        <dt>Hotel Name:</dt><dd>{row.hotel}</dd>
                        <dt>Rating:</dt><dd>{row.rating}</dd>
                        <dt>Address:</dt><dd>{row.address}</dd>
                        <dt>Location:</dt><dd>({row.latitude}, {row.longitude})</dd>
                        </dl>
                        """
    infos.append(info_box_template)
    
infos[0]

'<dl>\n                        <dt>Hotel Name:</dt><dd>Sheraton Khalidiya Hotel</dd>\n                        <dt>Rating:</dt><dd>3.8</dd>\n                        <dt>Address:</dt><dd>Zayed The First Street, Opposite KM Trading</dd>\n                        <dt>Location:</dt><dd>(24.4538352, 54.3774014)</dd>\n                        </dl>\n                        '

In [14]:
data = df.loc[:, ["latitude", "longitude"]]
hotel_data = hotel_full.loc[:, ["latitude", "longitude"]]

figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '0px',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type='SATELLITE')

heat = gmaps.heatmap_layer(data, weights=df.humidity)
fig.add_layer(heat)

markers = gmaps.marker_layer(hotel_data, info_box_content=infos)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(border='0px', height='600px', margin='0 auto 0 auto', padding='1px', width='800px')…